In [53]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import psutil

# Cargar el dataset de películas
movies_path = '../Data/processed_data/movies_dataset.parquet'
movies = pd.read_parquet(movies_path)

# Mostrar información básica del dataset
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  2655 non-null   object        
 1   budget                 25000 non-null  int64         
 2   genres                 25000 non-null  category      
 3   id                     25000 non-null  object        
 4   original_language      24998 non-null  object        
 5   popularity             25000 non-null  object        
 6   production_companies   25000 non-null  category      
 7   production_countries   25000 non-null  category      
 8   release_date           25000 non-null  datetime64[ns]
 9   revenue                25000 non-null  float64       
 10  runtime                24890 non-null  float64       
 11  spoken_languages       25000 non-null  object        
 12  status                 24946 non-null  object        
 13  t

In [54]:
# Crear una columna 'predictor' combinando información de las películas para generar recomendaciones
movies['predictor'] = (
    movies['genres'].astype(str).fillna('') + ' ' +
    movies['title'].fillna('') + ' ' +
    movies['overview_clean'].fillna('')
)

In [55]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer(min_df=4, max_df=0.85, ngram_range=(1, 2), max_features=40000, dtype=np.float32)
matriz_tfidf = vectorizer.fit_transform(movies['predictor'])

# Reducción de dimensionalidad con SVD
svd = TruncatedSVD(n_components=300, random_state=42)
matriz_reducida = svd.fit_transform(matriz_tfidf)

In [56]:
# Función para obtener el uso de memoria
def get_memory_usage():
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / (1024 * 1024)  # Convertir de bytes a MB

# Mostrar estadísticas de memoria y características
num_features = len(vectorizer.vocabulary_)
print(f"Uso de memoria: {get_memory_usage()} MB")
print(f"El vectorizador está utilizando {num_features} características.")
print(f"Dimensiones reducidas a {matriz_reducida.shape[1]} componentes.")

Uso de memoria: 636.90625 MB
El vectorizador está utilizando 31986 características.
Dimensiones reducidas a 300 componentes.


In [57]:
# Función de recomendación
def recomendacion(title):
    # Encuentra el índice de la película
    title = title.lower()
    idx = movies.index[movies['title'].str.lower() == title].tolist()
    if not idx:
        return "Película no encontrada"
    idx = idx[0]
    
    # Calcular la matriz de similitud del coseno
    sim_scores = cosine_similarity(matriz_reducida[idx].reshape(1, -1), matriz_reducida)
    
    # Obtén los puntajes de similitud para la película seleccionada
    sim_scores = list(enumerate(sim_scores[0]))
    
    # Ordena las películas basadas en la similitud
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Obtén los índices de las películas más similares
    movie_indices = [i[0] for i in sim_scores[1:6]]
    recomendaciones = movies['title'].iloc[movie_indices].tolist()
    
    return {"Recomendaciones": recomendaciones}

# Probar la función de recomendación
print(recomendacion('The Dark Knight'))

{'Recomendaciones': ['The Dark Knight Rises', 'Batman Begins', 'Batman Forever', 'Batman: The Dark Knight Returns, Part 1', 'Get the Gringo']}


In [58]:
# Verificar el uso de memoria después de cargar y procesar los datos
print(f"Uso de memoria: {get_memory_usage()} MB")

Uso de memoria: 637.32421875 MB


In [59]:
# Directorio de salida para guardar el modelo entrenado
directory = '../models/'
if not os.path.exists(directory):
    os.makedirs(directory)

In [60]:
# Guardar el modelo TF-IDF y la matriz reducida
joblib.dump(vectorizer, os.path.join(directory, 'vectorizer.pkl'))
joblib.dump(matriz_reducida, os.path.join(directory, 'matriz_reducida.pkl'))
print("Modelo guardado en:", directory)

Modelo guardado en: ../models/


In [61]:
# Cálculo del tamaño de los archivos guardados
vectorizer_path = os.path.join(directory, 'vectorizer.pkl')
matriz_reducida_path = os.path.join(directory, 'matriz_reducida.pkl')

# Función para obtener el tamaño de un archivo
def get_file_size(file_path):
    try:
        size = os.path.getsize(file_path)  # Obtener el tamaño en bytes
        return size / (1024 * 1024)  # Convertir a MB
    except FileNotFoundError:
        return "El archivo no existe."

# Obtener y mostrar el tamaño de los archivos
vectorizer_size = get_file_size(vectorizer_path)
matriz_reducida_size = get_file_size(matriz_reducida_path)

print(f"Tamaño del vectorizer: {vectorizer_size:.2f} MB")
print(f"Tamaño de la matriz reducida: {matriz_reducida_size:.2f} MB")

Tamaño del vectorizer: 1.08 MB
Tamaño de la matriz reducida: 28.61 MB
